# Analýza biomedicínských dat - bílá nebo černá magie?
***
**Mgr. Adéla Vrtková** (adela.vrtkova@vsb.cz)

Lékařská fakulta, Ostravská univerzita

Katedra aplikované matematiky, Fakulta elektrotechniky a informatiky, VŠB - Technická univerzita Ostrava
***
**Tento materiál byl vytvořen pro zvanou přednášku v rámci projektu CeBMI.**
***
**Zvláštní poděkování míří ke kolegům z Katedry aplikované matematiky, Fakulty elektrotechniky a informatiky, VŠB - Technické univerzity Ostrava - Ing. Martině Litschmannové, Ph.D. a Ing. Michalovi Bérešovi za jejich nápady a rady.**
***

In [ ]:
# Příprava prostředí
library(readxl)
library(ggplot2)
library(dplyr)
library(tidyr)

# Nastavení vykreslování grafů pro jupyter notebook
options(repr.plot.width = 10, repr.plot.height = 8)

#### Popis datového souboru
Kachexie označuje stav silné celkové sešlosti jedince spojené s úbytkem váhy a výraznou slabostí. Většinou je následkem vážných nádorových onemocnění nebo těžkých infekcí. V tomto kontextu vědci často zkoumají složení krve v závislosti na onemocnění pacienta a na výskytu kachexie.

V datovém souboru najdete údaje o onkologických pacientech s rakovinou plic s kachexií a bez projevů kachexie. Zároveň jsou v datech údaje i o třetí skupině – a to kontrolní skupině zdravých pacientů, tj. bez rakoviny i bez kachexie. U všech pacientů byla měřena hladina hormonu rezistinu (ng/ml).

U onkologických pacientů byla hladina rezistinu změřena při vstupní prohlídce a poté 3 měsíce po chemoterapii. U pacientů z kontrolní skupiny byl rezistin změřen pouze při vstupní prohlídce.

Skupina onkologických pacientů s kachexií je v datech kódována jako „RK“. Skupina onkologických pacientů bez kachexie je kódována „RBK“ a kontrolní skupina jako „KONT“. U onkologických pacientů je navíc uvedeno stádium rakoviny (na škále 0, I, II, III, IV).


In [ ]:
# Import dat, která jsou ve std. datovém formátu
data <- read_excel("data_kachexie.xlsx")

In [ ]:
head(data)

In [ ]:
tail(data)

## Vizualizace – mocná čarodějka
### Seznamte se, ggplot2

Základní fungování ggplot2 je pomocí vrstev.
***
Nejprve definujeme **"estetiku"** (aesthetics - aes):
- Důležitá část, kde specifikujeme proměnnou na ose x a/nebo na ose y.
- Lze ale i určit parametr, který ovlivní velikost (size) nebo barvu (color) vykreslených objektů (např. bodů).
- Dalšími parametry v estetice jsou - fill, linetype, label, shape a další...
***
Následuje určení **"geometrie"** (geometries - geom_???):
- Tato část definuje, jak se mají data znázornit.
- Např. jako body (geom_point), čáry (geom_line), krabicové grafy (geom_boxplot), sloupcové grafy (geom_bar),...
- Je třeba uvážit typ dat a na základě toho, jakou chceme informaci předat, zvolit geometrii.
- Různé "geom" lze i kombinovat, má-li to smysl.
***
V dalších vrstvách lze nastavovat:
- rozdělení na tzv. **"facets"** (mřížka grafů),
- zakomponování "statistiky" - vrstva **"statistics"** - např. přidání trendu, vykreslení průměru jako bodu apod.,
- vrstvu **"coordinates"** měřítka os, změnit je na logaritmické, apod.,
- změnit vzhled pomocí definovaných grafických témat - vrstva **"theme"** - případně si nastavit své vlastní.
    
https://www.rstudio.com/wp-content/uploads/2015/03/ggplot2-cheatsheet.pdf

In [ ]:
# Příprava "tidy" dat ze standardního datového formátu
# Sledovaný cíl: Analyzovat (vizualizovat) vývoj hladiny rezistinu od vstupní prohlídky k prohlídce po 3 měsících
data_tidy = 
        data %>% 
              pivot_longer(cols = c("Rezistin_vstup", "Rezistin_3M"), 
                           names_to = "Prohlidka", 
                           values_to = "Rezistin")

In [ ]:
head(data_tidy)

In [ ]:
tail(data_tidy)

In [ ]:
# Překódování nové proměnné Prohlidka dle potřeby
data_tidy$Prohlidka = factor(data_tidy$Prohlidka, 
                             levels = c("Rezistin_vstup", "Rezistin_3M"),
                             labels = c("Vstup","3 měsíce"))

Uvědomme si, že struktura *data_tidy* neodpovídá tomu, čemu říkáme standardní datový formát. Proto je vždy důležité vědět, co data znamenají a mít v nich nějaký identifikátor, pomocí kterého jsme schopni zpětně data zrekonstruovat do std. datového formátu, tzn. "spárovat" si údaje pro jednu stat. jednotku (zde je stat. jednotkou pacient).

### Síla krabicových grafů

In [ ]:
ggplot(data_tidy, aes(x = Prohlidka, y = Rezistin)) +
  geom_boxplot()+ 
  stat_summary(geom = "point", fun.y = "mean", colour = "red", size = 4)

In [ ]:
ggplot(data_tidy, aes(x = Prohlidka, y = Rezistin, fill = Skupina)) +
  geom_boxplot()

In [ ]:
ggplot(data_tidy, aes(x = Prohlidka, y = Rezistin, color = Skupina)) +
  geom_boxplot()

In [ ]:
ggplot(data_tidy, aes(x = interaction(Prohlidka, Skupina), y = Rezistin)) +
  geom_boxplot()

In [ ]:
ggplot(data_tidy, aes(x = Prohlidka, y = Rezistin)) +
  geom_boxplot(alpha = 0.5) +
  geom_line(aes(group = ID), size = 0.2)+
  geom_point(size = 0.8)+
  facet_wrap("Skupina", dir = "h")